In [28]:
from dotenv import load_dotenv
import pandas as pd

from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import PromptTemplate

import os

load_dotenv('.env')

True

In [29]:
llm = AzureChatOpenAI(azure_deployment=os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME_GPT3'), 
                                model_name=os.getenv('AZURE_OPENAI_MODEL_NAME_GPT3'),
                                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
                                azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                                api_version=os.getenv('AZURE_OPENAI_VERSION'))

In [30]:
annotations_df = pd.read_csv('../data/annotations.csv')
project_ids = annotations_df['ProjectID'].unique().tolist()
project_ids

[35, 34, 19, 36]

In [31]:
requirements_df = pd.read_csv('requirements_grouped.csv')
requirements_df.head()

,ProjectID,RequirementText
0,17,The system shall provide an option to withdraw...
1,19,The system must be easy to use by both manager...
2,20,All Web pages generated by the system shall be...
3,21,Programmers and application developers will ha...
4,24,The application should be connected to the Int...


### Generate Workflows for each of the SRS documents.

In [32]:
prompt = """
You are an expert business process consultant. 
You must come up with a workflow from the given software requirements that represents the logic of the process. 
You must respond with a list of lists, where each element of the inner list is a pair of nodes.
Do not provide any extra information.
You must use as few words as possible for each node.
Each node must represent one step in the workflow.
The steps in the graph must describe how the system works end to end.
The process must be representative of BPMN notation. 
If you create an end node, then that node should not have any children, ie, nodes that follow it.

For example, if the requirements say: 
'All Web pages generated by the system shall be fully downloadable in no more than 10 seconds over a 40KBps 
modem connection.', 'Responses to queries shall take no longer than 7 seconds to load onto the screen after the user submits the 
query.', 'The system shall display confirmation messages to users within 4 seconds after the user submits information to
the system.', 'All network transactions that involve financial information or personally identifiable information shall be 
encrypted.', 'Users shall be required to log in to the Cafeteria Ordering System for all operations except viewing a menu.', 'Patrons shall log in according to the restricted computer system access policy.', 'The system shall permit only cafeteria staff members who are on the list of authorized Menu Managers to create or edit menus.', 'Only users who have been authorized for home access to the corporate Intranet may use the COS from non-company locations.'

Then you would respond with: 
    [["Start", "Access System"],
    ["Access System", "View Menu"],
    ["View Menu", "Exclusive Gateway: Need to log in?"],
    ["Exclusive Gateway: Need to log in?", "User Login"],
    ["Exclusive Gateway: Need to log in?", "End"],
    ["User Login", "Verify Credentials"],
    ["Verify Credentials", "Exclusive Gateway: Company network?"],
    ["Exclusive Gateway: Company network?", "Successful Login and Redirection"],
    ["Exclusive Gateway: Company network?", "Home Access Authorization"],
    ["Home Access Authorization", "Exclusive Gateway: Authorized?"],
    ["Exclusive Gateway: Authorized?", "Successful Login and Redirection"],
    ["Exclusive Gateway: Authorized?", "End: Access Denied"],
    ["Successful Login and Redirection", "Exclusive Gateway: User role?"],
    ["Exclusive Gateway: User role?", "Menu Management"],
    ["Exclusive Gateway: User role?", "Order Placement"],
    ["Menu Management", "Save Changes"],
    ["Save Changes", "Display Confirmation"],
    ["Display Confirmation", "End"],
    ["Order Placement", "Submit Order"],
    ["Submit Order", "Encrypt Transactions"],
    ["Encrypt Transactions", "Process Order"],
    ["Process Order", "Display Confirmation"],
    ["Display Confirmation", "End"],
    ["Successful Login and Redirection", "End"],
    ["Order Placement", "End"],
    ["View Menu", "End"],
    ["Submit Query", "Process Query"],
    ["Process Query", "Load Query Results"],
    ["Load Query Results", "End"],
    ["Log Out", "Ensure Secure Termination"],
    ["Ensure Secure Termination", "End"]]

Now you must write out the process for:
{requirements}
"""

In [33]:
prompt = PromptTemplate(
    template=prompt,
    input_variables=["requirements"])

chain = prompt | llm  

In [37]:
output_df = None
for project in project_ids:
    requirements = " ".join(requirements_df.loc[requirements_df['ProjectID'] == project, 'RequirementText'])
    resp = chain.invoke({"requirements": requirements})
    lists = eval(resp.content)
    temp_df = pd.DataFrame(lists, columns=['from_node', 'to_node'])
    temp_df['ProjectID'] = project
    temp_df = temp_df[['ProjectID', 'from_node', 'to_node']]
    if output_df is None:
        output_df = temp_df
    else:
        output_df = pd.concat([output_df, temp_df], axis=0)

output_df.to_csv("../data/generated_graphs.csv", index=False)

In [38]:
output_df.head()

,ProjectID,from_node,to_node
0,35,Start,Display Logo and Basic Design
1,35,Display Logo and Basic Design,Attractive Design and Color
2,35,Attractive Design and Color,Comfortable User Experience
3,35,Attractive Design and Color,Reflect Seriousness of Environment
4,35,Display Logo and Basic Design,Manage Incoming Information
